# GLUT1 Analysis - Main Notebook

### 1. Preparing the data
Folder structure:
+ *Main Folder*
    + *Rawdata*
    + *Templates*
    + Animal_Scan_IDs.xlsx
    
    
- In the *Rawdata* folder each animal has a subfolder containing all the raw scans (simply the folders given by ParaVision)
- *Templates* folder contains templates for coregistration for each MR sequence (just pick one of the animals in the group). See examples.
- *Excel* file contains all animal ID's and their corresponding scan numbers for each modality (see example file).

In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
import MRIhelperfunctions as h
import MRIhelperfunctions
from MRIhelperfunctions import Settings
from imp import importlib
import fnmatch
import shutil
import papermill as pm
import glob


In [9]:
importlib.reload(h)
import MRIhelperfunctions as h
from MRIhelperfunctions import Settings

## Folder settings
Adjust these settings once for each study (namely location of rawdata folder and name of excel info file). These are constant across all animals and modalities

In [27]:
raw_folder = '/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata'   #<-- change this
main_folder = os.path.dirname(raw_folder)  # automatically uses one folder above rawdata folder as "main folder".

excel_filename = 'Animal_Scan_IDs.xlsx'  #<-- change this. Must be placed inside main folder


analysis_folder = os.path.join(main_folder,'Analysis')
notebookfolder = '/BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks'

folders = {}
folders.update({'main':main_folder, 'raw':raw_folder, 'analysis':analysis_folder, 'notebooks':notebookfolder, 'excel':excel_filename})
# folders = Settings(main = main_folder, raw = raw_folder, analysis = analysis_folder, notebooks = notebookfolder, excel = excel_filename)

# Check in output below if Excel file is correctly recognized
info = h.getinfo(folders)
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...


In [3]:
# use code below to locate folders of individual animals:
info.loc['folder','glut1_f10wks3']

'/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180607_133545_Glut1_2_6'

## Main loop

Runs the selected analyses (modules) for each individual animal.

- To turn a certain analysis on or off: comment/uncomment the line with *#*
- to run the loop only on a single/subset of animals: index *info.columns*, e.g. info.columns[1:2] would run only the 2nd animal
- If the analysis crashes after a couple of animals, use the option 'overwrite = False' to run loop again only on the animals that were not yet processed.


In [40]:
# for animal in ['glut1_f10wks2']:  # to run only one or subset of animals, enter here and comment out line below.
for animal in info.columns:   
    print('animal: ',animal)
    folders['animal']=info.loc['folder',animal]
    inputs = dict(folders=folders, animal=animal, info=info.to_json())
    
    
#     h.run_module(folders, 'Angiography', inputs)  # ~2-3 min

#     h.run_module(folders, 'Leakage', inputs, overwrite = True) # ~10-15 min

#     h.run_module(folders, 'BolusTracking', inputs) # ~8 min

    h.run_module(folders, 'Diamox', inputs, overwrite = True) # ~50 min
    h.pushover(f"Analysis for {animal} finished")

print('All complete!')
h.pushover(f"All analyses complete")


animal:  glut1_male1
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_101508_Glut1_2_2/Diamox_output.ipynb










  0%|          | 0/23 [00:00<?, ?it/s]









 13%|█▎        | 3/23 [00:01<00:09,  2.10it/s]









 22%|██▏       | 5/23 [00:01<00:06,  2.82it/s]









 35%|███▍      | 8/23 [00:01<00:03,  4.25it/s]









 43%|████▎     | 10/23 [00:53<01:09,  5.38s/it]









 48%|████▊     | 11/23 [00:55<01:00,  5.02s/it]









 57%|█████▋    | 13/23 [00:55<00:42,  4.28s/it]









 70%|██████▉   | 16/23 [1:30:51<39:45, 340.72s/it]









 91%|█████████▏| 21/23 [1:30:51<08:39, 259.60s/it]









 91%|█████████▏| 21/23 [1:31:05<08:40, 260.28s/it]









 96%|█████████▌| 22/23 [1:39:48<04:32, 272.21s/it]









100%|██████████| 23/23 [1:39:48<00:00, 260.38s/it]

animal:  glut1_male2
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180604_135437_Glut1_2_3/Diamox_output.ipynb










  0%|          | 0/23 [00:00<?, ?it/s]









 13%|█▎        | 3/23 [00:01<00:12,  1.54it/s]









 22%|██▏       | 5/23 [00:02<00:08,  2.21it/s]









 35%|███▍      | 8/23 [00:02<00:04,  3.32it/s]









 43%|████▎     | 10/23 [00:44<00:58,  4.48s/it]









 48%|████▊     | 11/23 [00:45<00:49,  4.14s/it]









 57%|█████▋    | 13/23 [00:45<00:35,  3.53s/it]









 70%|██████▉   | 16/23 [1:26:53<38:01, 325.87s/it]









 87%|████████▋ | 20/23 [1:26:54<13:02, 260.70s/it]









 87%|████████▋ | 20/23 [1:27:05<13:03, 261.25s/it]









 96%|█████████▌| 22/23 [1:38:46<04:29, 269.37s/it]









100%|██████████| 23/23 [1:38:46<00:00, 257.66s/it]

animal:  glut1_f10wks1
running Diamox analysis...


...

animal:  glut1_f6wks2
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180618_105806_Glut1_2_18/Diamox_output.ipynb










  0%|          | 0/23 [00:00<?, ?it/s]









  4%|▍         | 1/23 [00:00<00:13,  1.64it/s]









 13%|█▎        | 3/23 [00:01<00:13,  1.51it/s]









 22%|██▏       | 5/23 [00:02<00:08,  2.22it/s]









 35%|███▍      | 8/23 [00:02<00:04,  3.38it/s]









 43%|████▎     | 10/23 [00:45<00:58,  4.52s/it]









 48%|████▊     | 11/23 [00:46<00:50,  4.24s/it]









 57%|█████▋    | 13/23 [00:47<00:36,  3.62s/it]









 70%|██████▉   | 16/23 [56:41<24:48, 212.60s/it]









 87%|████████▋ | 20/23 [56:41<08:30, 170.09s/it]









 87%|████████▋ | 20/23 [56:59<08:32, 170.99s/it]









 96%|█████████▌| 22/23 [1:05:22<02:58, 178.27s/it]









100%|██████████| 23/23 [1:05:22<00:00, 170.52s/it]

animal:  glut1_f6wks1
running Diamox analysis...


Input Notebook:  /BigHDD/MR_DATA/GithubRepositories/CalciumfMRI/fMRI_notebooks/Diamox.ipynb
Output Notebook: /home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/20180618_084842_Glut1_2_17/Diamox_output.ipynb










  0%|          | 0/23 [00:00<?, ?it/s]









  4%|▍         | 1/23 [00:00<00:13,  1.65it/s]









 13%|█▎        | 3/23 [00:01<00:12,  1.59it/s]









 22%|██▏       | 5/23 [00:02<00:07,  2.27it/s]









 35%|███▍      | 8/23 [00:02<00:04,  3.47it/s]









 43%|████▎     | 10/23 [00:45<00:59,  4.57s/it]









 48%|████▊     | 11/23 [00:47<00:51,  4.33s/it]









 57%|█████▋    | 13/23 [00:48<00:36,  3.69s/it]









 65%|██████▌   | 15/23 [00:50<00:26,  3.35s/it]









 70%|██████▉   | 16/23 [1:07:55<29:43, 254.73s/it]









 91%|█████████▏| 21/23 [1:07:55<06:28, 194.09s/it]









 91%|█████████▏| 21/23 [1:08:15<06:30, 195.02s/it]









 96%|█████████▌| 22/23 [1:14:09<03:22, 202.27s/it]









100%|██████████| 23/23 [1:14:09<00:00, 193.47s/it]

All complete!


## Collecting outputs for group analysis

copies only relevant files of each animal in group folder

In [42]:
folders['group'] = h.folder_check_create(folders['main'], 'Groupstats')


for animal in info.columns:   
    folders['animal']=info.loc['folder',animal]

#     info, folders = h.copy_notebook_outputs(folders, animal, 'Angiography', info)

    info, folders = h.copy_notebook_outputs(folders, animal, 'BolusTracking', info)
    
    info, folders = h.copy_notebook_outputs(folders, animal, 'Diamox', info)
    
#     info, folders = h.copy_notebook_outputs(folders, animal, 'Leakage', info)
    
    
    # copying some underlay files to the folders
    template_file = os.path.join(folders['main'], 'Templates', 'BolusTracking.nii')    
    output = os.path.join(folders['group'], 'BolusTracking', 'Template.nii')
    shutil.copy(template_file, output)
    output = os.path.join(folders['group'], 'Diamox', 'Template.nii') # same scan settings as BolusTracking
    shutil.copy(template_file, output)
    
info

,glut1_male1,glut1_male2,glut1_f10wks1,glut1_f10wks2,glut1_f10wks3,glut1_f10wks4,glut1_f10wks5,glut1_f10wks6,glut1_f8wks1,glut1_f8wks2,glut1_f8wks3,glut1_f8wks4,glut1_f8wks5,glut1_f8wks6,glut1_f8wks7,glut1_f6wks2,glut1_f6wks1
scan,,,,,,,,,,,,,,,,,
genotype,1,1,1,2,1,2,2,1,1,1,2,2,2,2,1,1,1
Gd DCE,6,8,6,5,5,8,5,5,7,5,6,6,8,8,6,5,5
Angio pre,8,9,7,6,6,9,6,6,8,6,9,8,9,9,8,6,6
Angio post,12,12,11,9,9,13,10,11,12,11,12,12,16,13,12,9,10
Endorem DCE,11,11,10,8,8,12,9,10,10,10,11,11,15,12,11,8,9
Diamox,15,14,13,11,13,15,12,13,15,13,16,17,20,16,14,11,13
folder,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018060...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...,/home/aic/BigHDD/MR_DATA/GLUT1/Rawdata/2018061...
diamox_slope,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...
diamox_ttp,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...,/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_...


In [15]:
# to display location of individual files (first column name, then animal name)
info.loc['diamox_slope','glut1_male1']

'/home/aic/BigHDD/MR_DATA/GLUT1/Analysis/glut1_male1/Diamox2/diamox_slope_glut1_male1.nii'

## Create average & statistical maps

### Leakage

In [19]:
map_names = ['Leakage_max']   # prefixes of the output files

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_Leakage'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_Leakage'], map_type, group1, group2, blur=0.0)

created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Leakage/meanLeakage_max_Ttest.nii


### Bolus Tracking

In [45]:
map_names = ['CBF_map', 'CBV_bolus_map', 'MTT_map', 'CBV_steadystate_map']   # prefixes of the output files

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_BolusTracking'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_BolusTracking'], map_type, group1, group2, blur=0.2)

created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanCBF_map_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanCBV_bolus_map_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanMTT_map_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/BolusTracking/meanCBV_steadystate_map_Ttest.nii


### Diamox

In [44]:
map_names = ['diamox_ttp', 'diamox_slope', 'diamox_max']

for map_type in map_names:
    group1, group2 = h.select_for_group_average(folders['group_Diamox'],
                             info, map_type,
                             exclude='male',  #<-- excludes animals that contain this string in animal name
                             row = 'genotype')
    h.ttest_2groups(folders['group_Diamox'], map_type, group1, group2, blur=0.2)

created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Diamox/meandiamox_ttp_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Diamox/meandiamox_slope_Ttest.nii
created /home/aic/BigHDD/MR_DATA/GLUT1/Groupstats/Diamox/meandiamox_max_Ttest.nii


## Get ROI averages

In [ ]:
folders['bolustracking_roi'] = '/home/aic/BigHDD/MR_DATA/GLUT1/Templates/ROIs_BolusTracking.nii'


for animal in info.columns:
folders[''] = extract_roi